In [1]:
from __future__ import division
from __future__ import print_function

import sys, os, gc
import tensorflow as tf
import numpy as np
from sklearn import linear_model
from sklearn.metrics import roc_curve, auc


sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from models.neural_network import NeuralNetwork
from models.cnn import CNN
from models.util import *


#Seed used for choosing classes, training points, and test points.
SEED = 14

Using TensorFlow backend.


In [2]:
#Define params of model
input_shape = (28,28,1)
num_classes = 10
eps=0.3



In [3]:
#Load model from disk
model_name = 'MNIST'
model_save_path = '../trained_models/' + model_name + '-model.json'
weights_save_path = '../trained_models/' + model_name + 'weights'
model = CNN(model_name=model_name, dataset='mnist', seed=SEED)
print ('Total Model Params: %d' % model.num_params)
model.load_model(model_save_path, weights_save_path) 
#epochs = 50
#model.train(epochs=epochs)
#model.save_model(model_save_path, weights_save_path)    

Total Model Params: 3330314
Loaded model from disk


In [4]:
#Model Accuracy
print ('Model Accuracy: %.5f' % (model.model.evaluate(model.test_data, model.test_labels)[1]))

6055/6055 [==============================] - 1s 222us/step
Model Accuracy: 0.99306


In [5]:
#Get training samples
num_train_samples = 1000
data_indices = model.gen_rand_indices(low=0, high=model.train_data.shape[0], seed=SEED, num_samples=num_train_samples)
train_data = model.train_data[data_indices]
train_data_labels = model.train_labels[data_indices]
train_data_labels_int = np.argmax(train_data_labels, axis=1)

**GREYBOX ATTACKS**

In [6]:
num_test_samples_per_class = 100
num_test_samples = num_classes*num_test_samples_per_class

#Generate test points
test_indices = model.gen_rand_indices_all_classes(y=model.test_labels, seed=SEED, num_samples=num_test_samples_per_class)

#Get Regular, Noisy, FGSM, BIM, and CW test points
reg_data = model.test_data[test_indices]
fgsm_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='FGSM', eps=eps)
bim_a_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices], seed=SEED, perturbation='BIM-A', iterations=10, eps=eps)
bim_b_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices], seed=SEED, perturbation='BIM-B', iterations=10, eps=eps)
cw_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='CW', targeted=False, eps=eps)
df_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='DF', nb_candidate=num_classes)
jsma_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='JSMA')

In [7]:
#Reset tf.graph() as Cleverhans modifies the graph
tf.reset_default_graph()

#Reload the model and weights
model = CNN(model_name=model_name, dataset='mnist', seed=SEED)
model.load_model(model_save_path, weights_save_path)    


KeyboardInterrupt: 

In [ ]:
print ('Model Accuracy REG: %.5f' % (model.model.evaluate(reg_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy FGSM: %.5f' % (model.model.evaluate(fgsm_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy BIM-A: %.5f' % (model.model.evaluate(bim_a_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy BIM-B: %.5f' % (model.model.evaluate(bim_b_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy CW: %.5f' % (model.model.evaluate(cw_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy DF: %.5f' % (model.model.evaluate(df_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy JSMA: %.5f' % (model.model.evaluate(jsma_data,model.test_labels[test_indices])[1]))

In [ ]:
#Lets visualize samples from each attack
x_vis = np.random.choice(range(0,num_test_samples), 10)
print ('Regular: ')
visualize(reg_data[x_vis].reshape(-1,*input_shape), 10, '../figures/reg-cifar2')
print ('FGSM: ')
visualize(fgsm_data[x_vis].reshape(-1,*input_shape), 10, '../figures/fgsm-cifar2')
print ('BIM-A: ')
visualize(bim_a_data[x_vis].reshape(-1,*input_shape), 10, '../figures/bim-a-cifar2')
print ('BIM-B: ')
visualize(bim_b_data[x_vis].reshape(-1,*input_shape), 10, '../figures/bim-b-cifar2')
print ('CW: ')
visualize(cw_data[x_vis].reshape(-1,*input_shape), 10, '../figures/cw-cifar2')
print ('DF: ')
visualize(df_data[x_vis].reshape(-1,*input_shape), 10, '../figures/df-cifar2')
print ('JSMA: ')
visualize(jsma_data[x_vis].reshape(-1,*input_shape), 10, '../figures/jsma-cifar2')

In [ ]:
#Get predictions
reg_preds = model.model.predict(reg_data.reshape(-1,*input_shape))
fgsm_preds = model.model.predict(fgsm_data.reshape(-1,*input_shape))
bim_a_preds = model.model.predict(bim_a_data.reshape(-1,*input_shape))
bim_b_preds = model.model.predict(bim_b_data.reshape(-1,*input_shape))
cw_preds = model.model.predict(cw_data.reshape(-1,*input_shape))
df_preds = model.model.predict(df_data.reshape(-1,*input_shape))
jsma_preds = model.model.predict(jsma_data.reshape(-1,*input_shape))

#Convert preds to labels
reg_labels = preds_to_labels(reg_preds)
fgsm_labels = preds_to_labels(fgsm_preds)
bim_a_labels = preds_to_labels(bim_a_preds)
bim_b_labels = preds_to_labels(bim_b_preds)
cw_labels = preds_to_labels(cw_preds)
df_labels = preds_to_labels(df_preds)
jsma_labels = preds_to_labels(jsma_preds)

#Select Adversarial Points (i.e. points that lead to misclassification)
true_preds = np.argmax(model.test_labels[test_indices], axis=1)

#Check which points are actually adversarial and select those
fgsm_idx = np.where(np.argmax(fgsm_preds, axis=1) != true_preds)[0]
bim_a_idx = np.where(np.argmax(bim_a_preds, axis=1) != true_preds)[0]
bim_b_idx = np.where(np.argmax(bim_b_preds, axis=1) != true_preds)[0]
cw_idx = np.where(np.argmax(cw_preds, axis=1) != true_preds)[0]
df_idx = np.where(np.argmax(df_preds, axis=1) != true_preds)[0]
jsma_idx = np.where(np.argmax(jsma_preds, axis=1) != true_preds)[0]


#Filter data points to be used for similarity
fgsm_data_fil = fgsm_data[fgsm_idx]
bim_a_data_fil = bim_a_data[bim_a_idx]
bim_b_data_fil = bim_b_data[bim_b_idx]
cw_data_fil = cw_data[cw_idx]
df_data_fil = df_data[df_idx]
jsma_data_fil = jsma_data[jsma_idx]


#Filter labels to be used
fgsm_labels_fil = fgsm_labels[fgsm_idx]
bim_a_labels_fil = bim_a_labels[bim_a_idx]
bim_b_labels_fil = bim_b_labels[bim_b_idx]
cw_labels_fil = cw_labels[cw_idx]
df_labels_fil = df_labels[df_idx]
jsma_labels_fil = jsma_labels[jsma_idx]

In [ ]:
#Get distortion 
print ('FGSM: %.5f' % (avg_l2_dist(reg_data[fgsm_idx], fgsm_data_fil)))
print ('BIM-A: %.5f' % (avg_l2_dist(reg_data[bim_a_idx], bim_a_data_fil)))
print ('BIM-B: %.5f' % (avg_l2_dist(reg_data[bim_b_idx], bim_b_data_fil)))
print ('CW: %.5f' % (avg_l2_dist(reg_data[cw_idx], cw_data_fil)))
print ('DF: %.5f' % (avg_l2_dist(reg_data[df_idx], df_data_fil)))
print ('JSMA: %.5f' % (avg_l2_dist(reg_data[jsma_idx], jsma_data_fil)))

In [ ]:
#Get cosine similarity and norms
grads_train = model.get_gradients_wrt_params(train_data, train_data_labels)
grads_train = normalize(grads_train)
grads_reg_norms, cos_sim_reg = norms_and_cos(model, reg_data, reg_labels, grads_train)
grads_fgsm_norms, cos_sim_fgsm =norms_and_cos(model, fgsm_data_fil, fgsm_labels_fil, grads_train)
grads_bim_a_norms, cos_sim_bim_a = norms_and_cos(model, bim_a_data_fil, bim_a_labels_fil, grads_train)
grads_bim_b_norms , cos_sim_bim_b= norms_and_cos(model, bim_b_data_fil, bim_b_labels_fil, grads_train)
grads_cw_norms, cos_sim_cw = norms_and_cos(model, cw_data_fil, cw_labels_fil, grads_train)
grads_df_norms, cos_sim_df = norms_and_cos(model, df_data_fil, df_labels_fil, grads_train)
grads_jsma_norms, cos_sim_jsma = norms_and_cos(model, jsma_data_fil, jsma_labels_fil, grads_train)

**LOGISTIC REGRESSION FOR GREYBOX**

In [ ]:
#Train a logistic regression classifier on the data. We only train on gray box attack points. 
#Due to lack of data (computationally expensive to compute attack points), we use 95% of data to train and 5% to test

#Select training and test indices
np.random.seed(SEED)
train_pct = .95
reg_train_idx = np.random.choice(np.arange(num_test_samples), int(train_pct*num_test_samples), replace=False)
reg_test_idx = get_test_from_train_idx(np.arange(num_test_samples), reg_train_idx)
fgsm_train_idx = np.random.choice(np.arange(len(fgsm_idx)), int(len(fgsm_idx)*train_pct), replace=False)
fgsm_test_idx = get_test_from_train_idx(np.arange(len(fgsm_idx)), fgsm_train_idx)
bim_a_train_idx = np.random.choice(np.arange(len(bim_a_idx)), int(len(bim_a_idx)*train_pct), replace=False)
bim_a_test_idx = get_test_from_train_idx(np.arange(len(bim_a_idx)), bim_a_train_idx)
bim_b_train_idx =np.random.choice(np.arange(len(bim_b_idx)), int(len(bim_b_idx)*train_pct), replace=False)
bim_b_test_idx = get_test_from_train_idx(np.arange(len(bim_b_idx)), bim_b_train_idx)
jsma_train_idx =np.random.choice(np.arange(len(jsma_idx)), int(len(jsma_idx)*train_pct), replace=False)
jsma_test_idx = get_test_from_train_idx(np.arange(len(jsma_idx)), jsma_train_idx)
cw_train_idx = np.random.choice(np.arange(len(cw_idx)), int(len(cw_idx)*train_pct), replace=False)
cw_test_idx = get_test_from_train_idx(np.arange(len(cw_idx)), cw_train_idx)
df_train_idx = np.random.choice(np.arange(len(df_idx)), int(len(df_idx)*train_pct), replace=False)
df_test_idx = get_test_from_train_idx(np.arange(len(df_idx)), df_train_idx)



# Set up training and test data for logistic regression
train_data = np.concatenate((cos_sim_reg[reg_train_idx], 
                             cos_sim_fgsm[fgsm_train_idx],
                             cos_sim_bim_a[bim_a_train_idx],
                             cos_sim_bim_b[bim_b_train_idx], 
                             cos_sim_jsma[jsma_train_idx], 
                             cos_sim_cw[cw_train_idx], 
                             cos_sim_df[df_train_idx]),axis=0)
train_labels = np.concatenate((np.zeros(len(reg_train_idx)), 
                               np.ones(len(fgsm_train_idx)),
                               np.ones(len(bim_a_train_idx)),
                               np.ones(len(bim_b_train_idx)),
                               np.ones(len(jsma_train_idx)),
                               np.ones(len(cw_train_idx)),
                               np.ones(len(df_train_idx))),axis=0)

test_data = np.concatenate((cos_sim_reg[reg_test_idx], 
                             cos_sim_fgsm[fgsm_test_idx],
                             cos_sim_bim_a[bim_a_test_idx],
                             cos_sim_bim_b[bim_b_test_idx], 
                             cos_sim_jsma[jsma_test_idx], 
                             cos_sim_cw[cw_test_idx], 
                             cos_sim_df[df_test_idx]),axis=0)

test_labels = np.concatenate((np.zeros(len(reg_test_idx)), 
                               np.ones(len(fgsm_test_idx)),
                               np.ones(len(bim_a_test_idx)),
                               np.ones(len(bim_b_test_idx)),
                               np.ones(len(jsma_test_idx)),
                               np.ones(len(cw_test_idx)),
                               np.ones(len(df_test_idx))),axis=0)

In [ ]:
#Fit the data
logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(train_data, train_labels)

#Get Accuracy for each attack type
fgsm_acc = logreg.score(cos_sim_fgsm[fgsm_test_idx], np.ones(len(fgsm_test_idx)))
bim_a_acc = logreg.score(cos_sim_bim_a[bim_a_test_idx], np.ones(len(bim_a_test_idx)))
bim_b_acc = logreg.score(cos_sim_bim_b[bim_b_test_idx], np.ones(len(bim_b_test_idx)))
jsma_acc = logreg.score(cos_sim_jsma[jsma_test_idx], np.ones(len(jsma_test_idx)))
cw_acc = logreg.score(cos_sim_cw[cw_test_idx], np.ones(len(cw_test_idx)))
df_acc = logreg.score(cos_sim_df[df_test_idx], np.ones(len(df_test_idx)))

#Get Total accuracy
total_acc = logreg.score(test_data, test_labels)

print ('FGSM Detection Acc: %.5f' % (fgsm_acc))
print ('BIM-A Detection Acc: %.5f' % (bim_a_acc))
print ('BIM-B Detection Acc: %.5f' % (bim_b_acc))
print ('JSMA Detection Acc: %.5f' % (jsma_acc))
print ('CW Detection Acc: %.5f' % (cw_acc))
print ('DF Detection Acc: %.5f' % (df_acc))
print ('TOTAL Detection Acc: %.5f' %(total_acc))


In [ ]:
#Plot ROC for the entire test dataset
probs = logreg.predict_proba(test_data)
fpr, tpr, _ = roc_curve(test_labels, probs[:, 1])
roc_auc = auc(fpr, tpr)

# Plot all ROC curves
plt.figure()
plt.plot(fpr, tpr,
         label='ROC curve (area = {0:0.2f})'
               ''.format(roc_auc),
         color='blue', linewidth=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.savefig('cifar.eps', format='eps', dpi=1000)
plt.show()


**THRESHOLDING FOR GREYBOX**

In [ ]:
#Ratio of perturbed samples having cos sim greater
print ('FGSM:  %.4f' % ( comp_cos(cos_sim_fgsm, cos_sim_reg[fgsm_idx])))
print ('BIM-A:  %.4f' % ( comp_cos(cos_sim_bim_a, cos_sim_reg[bim_a_idx])))
print ('BIM-B:  %.4f' % ( comp_cos(cos_sim_bim_b, cos_sim_reg[bim_b_idx])))
print ('CW: %.4f' % ( comp_cos(cos_sim_cw, cos_sim_reg[cw_idx])))
print ('DF: %.4f' % ( comp_cos(cos_sim_df, cos_sim_reg[df_idx])))
print ('JSMA: %.4f' % ( comp_cos(cos_sim_jsma, cos_sim_reg[jsma_idx])))

In [ ]:
#Ratio of perturbed samples having norm greater
print ('FGSM:  %.4f' % ( comp_norm(grads_fgsm_norms, grads_reg_norms[fgsm_idx])))
print ('BIM-A:  %.4f' % ( comp_norm(grads_bim_a_norms, grads_reg_norms[bim_a_idx])))
print ('BIM-B:  %.4f' % ( comp_norm(grads_bim_b_norms, grads_reg_norms[bim_b_idx])))
print ('CW: %.4f' % ( comp_norm(grads_cw_norms, grads_reg_norms[cw_idx])))
print ('DF: %.4f' % ( comp_norm(grads_df_norms, grads_reg_norms[df_idx])))
print ('JSMA: %.4f' % ( comp_norm(grads_jsma_norms, grads_reg_norms[jsma_idx])))

In [ ]:
#Separate Using Cos Sim
eta = 0.7
print ('Regular: %.4f' % ( greater_cos(cos_sim_reg, eta)))
print ('FGSM:  %.4f' % ( greater_cos(cos_sim_fgsm, eta)))
print ('BIM-A:  %.4f' % ( greater_cos(cos_sim_bim_a, eta)))
print ('BIM-B:  %.4f' % ( greater_cos(cos_sim_bim_b, eta)))
print ('CW: %.4f' % ( greater_cos(cos_sim_cw, eta)))
print ('DF: %.4f' % ( greater_cos(cos_sim_df, eta)))
print ('JSMA: %.4f' % ( greater_cos(cos_sim_jsma, eta)))

In [ ]:
#Separate using just norm
gamma = .02
print ('Regular: %.4f' % ( smaller_norm(grads_reg_norms, gamma)))
print ('FGSM:  %.4f' % ( smaller_norm(grads_fgsm_norms, gamma)))
print ('BIM-A:  %.4f' % ( smaller_norm(grads_bim_a_norms, gamma)))
print ('BIM-B:  %.4f' % ( smaller_norm(grads_bim_b_norms, gamma)))
print ('CW: %.4f' % ( smaller_norm(grads_cw_norms, gamma)))
print ('DF: %.4f' % ( smaller_norm(grads_df_norms, gamma)))
print ('JSMA: %.4f' % ( smaller_norm(grads_jsma_norms, gamma)))


In [ ]:
#Use both cos and norm
print ('Regular: %.4f' % ( cos_and_norm_sep(cos_sim_reg, grads_reg_norms, eta, gamma)))
print ('FGSM:  %.4f' % ( cos_and_norm_sep(cos_sim_fgsm, grads_fgsm_norms, eta, gamma)))
print ('BIM-A:  %.4f' % ( cos_and_norm_sep(cos_sim_bim_a, grads_bim_a_norms, eta, gamma)))
print ('BIM-B:  %.4f' % ( cos_and_norm_sep(cos_sim_bim_b, grads_bim_b_norms, eta, gamma)))
print ('CW: %.4f' % ( cos_and_norm_sep(cos_sim_cw, grads_cw_norms, eta, gamma)))
print ('DF: %.4f' % ( cos_and_norm_sep(cos_sim_df, grads_df_norms, eta, gamma)))
print ('JSMA: %.4f' % ( cos_and_norm_sep(cos_sim_jsma, grads_jsma_norms, eta, gamma)))

**WHITEBOX ATTACKS**

In [ ]:
#Guide Indices for P1 attacks
guide_indices_reg = list()
reg_filter = np.arange(num_test_samples)
np.random.seed(SEED)
#Generate guide images using max cosine similarity images
for i in range(num_test_samples):
    guide_img_idx = get_guide_idx(model,idx_filter=reg_filter,data_indices=data_indices,cos_sim=cos_sim_reg, idx=i)
    guide_indices_reg.append(guide_img_idx)

In [ ]:
#1 Phase Attacks
fgsm_wb_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='FGSM-WB', eps=eps, x_tar=model.train_data[guide_indices_reg], y_tar = model.train_labels[guide_indices_reg])
bim_a_wb_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='BIM-A-WB', eps=eps, x_tar=model.train_data[guide_indices_reg], y_tar = model.train_labels[guide_indices_reg])
bim_b_wb_data = model.generate_perturbed_data(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='BIM-B-WB', eps=eps, x_tar=model.train_data[guide_indices_reg], y_tar = model.train_labels[guide_indices_reg])

In [ ]:
#Reset tf.graph() as Cleverhans modifies the graph
tf.reset_default_graph()

#Reload the model and weights
model = CNN(model_name=model_name, dataset='mnist', seed=SEED)
model.load_model(model_save_path, weights_save_path)    



In [ ]:
#1 Phase CW Attack
p1_cw_data = model.generate_perturbed_data(model.test_data[test_indices], y_tar=model.train_labels[guide_indices_reg],seed=SEED, perturbation='CW', targeted=True, x_tar = model.train_data[guide_indices_reg], use_cos_norm_reg=True, eps=eps)

In [ ]:
#Reset tf.graph() as Cleverhans modifies the graph
tf.reset_default_graph()

#Reload the model and weights
model = CNN(model_name=model_name, dataset='mnist', seed=SEED)
model.load_model(model_save_path, weights_save_path)    


In [ ]:
#WB Attacks - Generate Guide indices for FGSM, BIM-A, BIM-B, and CW for P2 attack
guide_indices_fgsm = list()
guide_indices_bim_a = list()
guide_indices_bim_b = list()
guide_indices_cw = list()
np.random.seed(SEED)
#Generate guide images using max cosine similarity images
for i in range(num_test_samples):
    
    #Only use misclassified images
    if i not in fgsm_idx:
        continue
    
    #Choose the max similarity image as the guide image
    else:
        guide_img_idx_fgsm = get_guide_idx(model,idx_filter=fgsm_idx,cos_sim=cos_sim_fgsm,data_indices=data_indices, idx=i)
        
    if i not in bim_a_idx:
        continue
    
    else:
        guide_img_idx_bim_a = get_guide_idx(model,idx_filter=bim_a_idx,cos_sim=cos_sim_bim_a,data_indices=data_indices, idx=i)  
    
    if i not in bim_b_idx:
        continue
    
    else:
        guide_img_idx_bim_b = get_guide_idx(model,idx_filter=bim_b_idx,cos_sim=cos_sim_bim_b,data_indices=data_indices, idx=i) 
            
    if i not in cw_idx:
        continue
    
    else:
        guide_img_idx_cw = get_guide_idx(model,idx_filter=cw_idx,cos_sim=cos_sim_cw,data_indices=data_indices, idx=i)  

        
    guide_indices_fgsm.append(guide_img_idx_fgsm)
    guide_indices_bim_a.append(guide_img_idx_bim_a)
    guide_indices_bim_b.append(guide_img_idx_bim_b)
    guide_indices_cw.append(guide_img_idx_cw)



In [ ]:
#CW Phase 2: Optimize for higher cosine sim and smaller norm of gradient vector
p2_cw_data = model.generate_perturbed_data(cw_data_fil, y_tar=cw_labels_fil, seed=SEED, perturbation='CW', targeted=True, x_tar = model.train_data[guide_indices_cw], use_cos_norm_reg=True, eps=eps)

In [ ]:
#Reset tf.graph() as Cleverhans modifies the graph
tf.reset_default_graph()

#Reload the model and weights
model = CNN(model_name=model_name, dataset='mnist', seed=SEED)
model.load_model(model_save_path, weights_save_path)    


In [ ]:
#Whitebox CW 2 Phase attack with FGSM, BIM-a/b guide images
p2_cw_fgsm_data = model.generate_perturbed_data(fgsm_data_fil, y_tar=fgsm_labels_fil,seed=SEED, perturbation='CW', targeted=True, x_tar = model.train_data[guide_indices_fgsm], use_cos_norm_reg=True, eps=eps)
p2_cw_bim_a_data = model.generate_perturbed_data(bim_a_data_fil, y_tar=bim_a_labels_fil,seed=SEED, perturbation='CW', targeted=True, x_tar = model.train_data[guide_indices_bim_a], use_cos_norm_reg=True, eps=eps)
p2_cw_bim_b_data = model.generate_perturbed_data(bim_b_data_fil, y_tar=bim_a_labels_fil,seed=SEED, perturbation='CW', targeted=True, x_tar = model.train_data[guide_indices_bim_b], use_cos_norm_reg=True, eps=eps)

In [ ]:
#Reset tf.graph() as Cleverhans modifies the graph
tf.reset_default_graph()

#Reload the model and weights
model = CNN(model_name=model_name, dataset='mnist', seed=SEED)
model.load_model(model_save_path, weights_save_path)    



In [ ]:
print ('Model Accuracy FGSM-WB: %.5f' % (model.model.evaluate(fgsm_wb_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy BIM-A WB: %.5f' % (model.model.evaluate(bim_a_wb_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy BIM-B WB: %.5f' % (model.model.evaluate(bim_b_wb_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy P1 CW: %.5f' % (model.model.evaluate(p1_cw_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy P2 CW: %.5f' % (model.model.evaluate(p2_cw_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy P2 FGSM: %.5f' % (model.model.evaluate(p2_cw_fgsm_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy P2 BIM-A: %.5f' % (model.model.evaluate(p2_cw_bim_a_data,model.test_labels[test_indices])[1]))
print ('Model Accuracy P2 BIM-B %.5f' % (model.model.evaluate(p2_cw_bim_b_data,model.test_labels[test_indices])[1]))

In [ ]:
#Lets visualize one sample from each dataset
x_vis = np.random.choice(range(0,num_test_samples), 10)
print ('Regular: ')
visualize(reg_data[x_vis].reshape(-1,*input_shape), 10,)
print ('FGSM-WB: ')
visualize(fgsm_wb_data[x_vis].reshape(-1,*input_shape), 10, '../figures/fgsm-wb-cifar2')
print ('BIM-A-WB: ')
visualize(bim_a_wb_data[x_vis].reshape(-1,*input_shape), 10, '../figures/bim-a-wb-cifar2')
print ('BIM-B-WB: ')
visualize(bim_b_wb_data[x_vis].reshape(-1,*input_shape), 10, '../figures/bim-b-wb-cifar2')
print ('P1-CW: ')
visualize(p1_cw_data[x_vis].reshape(-1,*input_shape), 10, '../figures/p1-cw-cifar2')
print ('P2-CW: ')
visualize(p2_cw_data[x_vis].reshape(-1,*input_shape), 10, '../figures/p2-cw-cifar2')
print ('P2-FGSM: ')
visualize(p2_cw_fgsm_data[x_vis].reshape(-1,*input_shape), 10, '../figures/p2-fgsm-cifar2')
print ('P2-BIM-A')
visualize(p2_cw_bim_a_data[x_vis].reshape(-1,*input_shape), 10, '../figures/p2-bim-a-cifar2')
print ('P2-BIM-B')
visualize(p2_cw_bim_b_data[x_vis].reshape(-1,*input_shape), 10, '../figures/p2-bim-b-cifar2')


In [ ]:
#Get predictions
fgsm_wb_preds = model.model.predict(fgsm_wb_data.reshape(-1,*input_shape))
bim_a_wb_preds = model.model.predict(bim_a_wb_data.reshape(-1,*input_shape))
bim_b_wb_preds = model.model.predict(bim_a_wb_data.reshape(-1,*input_shape))
p1_cw_preds = model.model.predict(p1_cw_data.reshape(-1,*input_shape))
p2_cw_preds = model.model.predict(p2_cw_data.reshape(-1,*input_shape))
p2_cw_fgsm_preds = model.model.predict(p2_cw_fgsm_data.reshape(-1,*input_shape))
p2_cw_bim_a_preds = model.model.predict(p2_cw_bim_a_data.reshape(-1,*input_shape))
p2_cw_bim_b_preds = model.model.predict(p2_cw_bim_b_data.reshape(-1,*input_shape))


In [ ]:
#Convert preds to labels
fgsm_wb_labels = preds_to_labels(fgsm_wb_preds)
bim_a_wb_labels = preds_to_labels(bim_a_wb_preds)
bim_b_wb_labels = preds_to_labels(bim_b_wb_preds)
p1_cw_labels = preds_to_labels(p1_cw_preds)
p2_cw_labels = preds_to_labels(p2_cw_preds)
p2_cw_fgsm_labels = preds_to_labels(p2_cw_fgsm_preds)
p2_cw_bim_a_labels = preds_to_labels(p2_cw_bim_a_preds)
p2_cw_bim_b_labels = preds_to_labels(p2_cw_bim_b_preds)



In [ ]:
#Check which points are actually adversarial and select those
fgsm_wb_idx =  np.where(np.argmax(fgsm_wb_preds, axis=1) != true_preds)[0]
bim_a_wb_idx = np.where(np.argmax(bim_a_wb_preds, axis=1) != true_preds)[0]
bim_b_wb_idx = np.where(np.argmax(bim_b_wb_preds, axis=1) != true_preds)[0]
p1_cw_idx = np.where(np.argmax(p1_cw_preds, axis=1) != true_preds)[0]
p2_cw_idx = np.where(np.argmax(p2_cw_preds, axis=1) != true_preds)[0]
p2_cw_fgsm_idx = np.where(np.argmax(p2_cw_fgsm_preds, axis=1) != true_preds)[0]
p2_cw_bim_a_idx = np.where(np.argmax(p2_cw_bim_a_preds, axis=1) != true_preds)[0]
p2_cw_bim_b_idx = np.where(np.argmax(p2_cw_bim_b_preds, axis=1) != true_preds)[0]

#Filter data points to be used for similarity
fgsm_wb_data_fil = fgsm_wb_data[fgsm_wb_idx]
bim_a_wb_data_fil = bim_a_wb_data[bim_a_wb_idx]
bim_b_wb_data_fil = bim_b_wb_data[bim_b_wb_idx]
p1_cw_data_fil = p1_cw_data[p1_cw_idx]
p2_cw_data_fil = p2_cw_data[p2_cw_idx]
p2_cw_fgsm_data_fil = p2_cw_fgsm_data[p2_cw_fgsm_idx]
p2_cw_bim_a_data_fil = p2_cw_bim_a_data[p2_cw_bim_a_idx]
p2_cw_bim_b_data_fil = p2_cw_bim_b_data[p2_cw_bim_b_idx]

#Filter labels to be used
fgsm_wb_labels_fil = fgsm_wb_labels[fgsm_wb_idx]
bim_a_wb_labels_fil = bim_a_wb_labels[bim_a_wb_idx]
bim_b_wb_labels_fil = bim_b_wb_labels[bim_b_wb_idx]
p1_cw_labels_fil = p1_cw_labels[p1_cw_idx]
p2_cw_labels_fil = p2_cw_labels[p2_cw_idx]
p2_cw_fgsm_labels_fil = p2_cw_fgsm_labels[p2_cw_fgsm_idx]
p2_cw_bim_a_labels_fil = p2_cw_bim_a_labels[p2_cw_bim_a_idx]
p2_cw_bim_b_labels_fil = p2_cw_bim_b_labels[p2_cw_bim_b_idx]

In [ ]:
#Get distortion (Prints nan if no sample was adversarial)
print ('FGSM-WB: %.5f' % (avg_l2_dist(reg_data[fgsm_wb_idx], fgsm_wb_data_fil)))
print ('BIM-A-WB: %.5f' % (avg_l2_dist(reg_data[bim_a_wb_idx], bim_a_wb_data_fil)))
print ('BIM-B-WB: %.5f' % (avg_l2_dist(reg_data[bim_b_wb_idx], bim_b_wb_data_fil)))
print ('P1-CW: %.5f' % (avg_l2_dist(reg_data[p1_cw_idx], p1_cw_data_fil)))
print ('P2-CW: %.5f' % (avg_l2_dist(reg_data[p2_cw_idx], p2_cw_data_fil)))
print ('P2-CW FGSM: %.5f' % (avg_l2_dist(reg_data[p2_cw_fgsm_idx], p2_cw_fgsm_data_fil)))
print ('P2-CW BIM A: %.5f' % (avg_l2_dist(reg_data[p2_cw_bim_a_idx], p2_cw_bim_a_data_fil)))
print ('P2-CW BIM B: %.5f' % (avg_l2_dist(reg_data[p2_cw_bim_b_idx], p2_cw_bim_b_data_fil)))

In [ ]:
#Get cosine similarity and norms
grads_fgsm_wb_norms, cos_sim_fgsm_wb = norms_and_cos(model, fgsm_wb_data_fil, fgsm_wb_labels_fil, grads_train)
grads_bim_a_wb_norms, cos_sim_bim_a_wb = norms_and_cos(model, bim_a_wb_data_fil, bim_a_wb_labels_fil, grads_train)
grads_bim_b_wb_norms, cos_sim_bim_b_wb = norms_and_cos(model, bim_b_wb_data_fil, bim_b_wb_labels_fil, grads_train)
grads_p1_cw_norms, cos_sim_p1_cw = norms_and_cos(model, p1_cw_data_fil, p1_cw_labels_fil, grads_train)
grads_p2_cw_norms, cos_sim_p2_cw = norms_and_cos(model, p2_cw_data_fil, p2_cw_labels_fil, grads_train)
grads_p2_cw_fgsm_norms, cos_sim_p2_cw_fgsm = norms_and_cos(model, p2_cw_fgsm_data_fil, p2_cw_fgsm_labels_fil, grads_train)
grads_p2_cw_bim_a_norms, cos_sim_p2_cw_bim_a = norms_and_cos(model, p2_cw_bim_a_data_fil, p2_cw_bim_a_labels_fil, grads_train)
grads_p2_cw_bim_b_norms, cos_sim_p2_cw_bim_b = norms_and_cos(model, p2_cw_bim_b_data_fil, p2_cw_bim_b_labels_fil, grads_train)

**THRESHOLDING FOR WHITEBOX**

In [ ]:
#Ratio of perturbed samples having cos sim smaller
print ('FGSM-WB:  %.4f' % ( comp_cos(cos_sim_fgsm_wb, cos_sim_reg[fgsm_wb_idx])))
print ('BIM-A-WB:  %.4f' % ( comp_cos(cos_sim_bim_a_wb, cos_sim_reg[bim_a_wb_idx])))
print ('BIM-B-WB:  %.4f' % ( comp_cos(cos_sim_bim_b_wb, cos_sim_reg[bim_b_wb_idx])))
print ('1 Phase CW:  %.4f' % ( comp_cos(cos_sim_p1_cw, cos_sim_reg[p1_cw_idx])))
print ('2 Phase CW:  %.4f' % ( comp_cos(cos_sim_p2_cw, cos_sim_reg[p2_cw_idx])))
print ('2 Phase CW FGSM:  %.4f' % ( comp_cos(cos_sim_p2_cw_fgsm, cos_sim_reg[p2_cw_fgsm_idx])))
print ('2 Phase CW BIM A:  %.4f' % ( comp_cos(cos_sim_p2_cw_bim_a, cos_sim_reg[p2_cw_bim_a_idx])))
print ('2 Phase CW BIM B:  %.4f' % ( comp_cos(cos_sim_p2_cw_bim_b, cos_sim_reg[p2_cw_bim_b_idx])))


In [ ]:
#Ratio of perturbed samples having norm greater
print ('FGSM-WB:  %.4f' % ( comp_norm(grads_fgsm_wb_norms, grads_reg_norms[fgsm_wb_idx])))
print ('BIM-A-WB:  %.4f' % ( comp_norm(grads_bim_a_wb_norms, grads_reg_norms[bim_a_wb_idx])))
print ('BIM-B-WB:  %.4f' % ( comp_norm(grads_bim_b_wb_norms, grads_reg_norms[bim_b_wb_idx])))
print ('1 Phase CW: %.4f' % ( comp_norm(grads_p1_cw_norms, grads_reg_norms[p1_cw_idx])))
print ('2 Phase CW: %.4f' % ( comp_norm(grads_p2_cw_norms, grads_reg_norms[p2_cw_idx])))
print ('2 Phase CW FGSM: %.4f' % ( comp_norm(grads_p2_cw_fgsm_norms, grads_reg_norms[p2_cw_fgsm_idx])))
print ('2 Phase CW BIM A: %.4f' % ( comp_norm(grads_p2_cw_bim_a_norms, grads_reg_norms[p2_cw_bim_a_idx])))
print ('2 Phase CW BIM B: %.4f' % ( comp_norm(grads_p2_cw_bim_b_norms, grads_reg_norms[p2_cw_bim_b_idx])))


In [ ]:
#Separate Using Cos Sim
print ('REG:  %.4f' % ( greater_cos(cos_sim_reg, eta)))
print ('FGSM-WB:  %.4f' % ( greater_cos(cos_sim_fgsm_wb, eta)))
print ('BIM-A-WB:  %.4f' % ( greater_cos(cos_sim_bim_a_wb, eta)))
print ('BIM-B-WB:  %.4f' % ( greater_cos(cos_sim_bim_b_wb, eta)))
print ('1 Phase CW:  %.4f' % ( greater_cos(cos_sim_p1_cw, eta)))        
print ('2 Phase CW:  %.4f' % ( greater_cos(cos_sim_p2_cw, eta)))
print ('2 Phase CW FGSM:  %.4f' % ( greater_cos(cos_sim_p2_cw_fgsm, eta)))       
print ('2 Phase CW BIM-A:  %.4f' % ( greater_cos(cos_sim_p2_cw_bim_a, eta)))      
print ('2 Phase CW BIM-B:  %.4f' % ( greater_cos(cos_sim_p2_cw_bim_b, eta)))



In [ ]:
#Separate using just norm
print ('Regular: %.4f' % ( smaller_norm(grads_reg_norms, gamma)))
print ('FGSM-WB:  %.4f' % ( smaller_norm(grads_fgsm_wb_norms, gamma)))
print ('BIM-A-WB:  %.4f' % ( smaller_norm(grads_bim_a_wb_norms, gamma)))
print ('BIM-B-WB:  %.4f' % ( smaller_norm(grads_bim_b_wb_norms, gamma)))
print ('1 Phase CW:  %.4f' % ( smaller_norm(grads_p1_cw_norms, gamma)))        
print ('2 Phase CW:  %.4f' % ( smaller_norm(grads_p2_cw_norms, gamma)))
print ('2 Phase CW FGSM:  %.4f' % ( smaller_norm(grads_p2_cw_fgsm_norms, gamma)))       
print ('2 Phase CW BIM-A:  %.4f' % ( smaller_norm(grads_p2_cw_bim_a_norms, gamma)))      
print ('2 Phase CW BIM-B:  %.4f' % ( smaller_norm(grads_p2_cw_bim_b_norms, gamma)))

In [ ]:
#Use both cos and norm
print ('Regular: %.4f' % ( cos_and_norm_sep(cos_sim_reg, grads_reg_norms, eta, gamma)))
print ('FGSM-WB:  %.4f' % ( cos_and_norm_sep(cos_sim_fgsm_wb, grads_fgsm_wb_norms, eta, gamma)))
print ('BIM-A-WB:  %.4f' % ( cos_and_norm_sep(cos_sim_bim_a_wb, grads_bim_a_wb_norms, eta, gamma)))
print ('BIM-B-WB:  %.4f' % ( cos_and_norm_sep(cos_sim_bim_b_wb, grads_bim_b_wb_norms, eta, gamma)))
print ('1 Phase CW: %.4f' % ( cos_and_norm_sep(cos_sim_p1_cw, grads_p1_cw_norms, eta, gamma)))
print ('2 Phase CW: %.4f' % ( cos_and_norm_sep(cos_sim_p2_cw, grads_p2_cw_norms, eta, gamma)))
print ('2 Phase CW FGSM: %.4f' % ( cos_and_norm_sep(cos_sim_p2_cw_fgsm, grads_p2_cw_fgsm_norms, eta, gamma)))
print ('2 Phase CW BIM A: %.4f' % ( cos_and_norm_sep(cos_sim_p2_cw_bim_a, grads_p2_cw_bim_a_norms, eta, gamma)))
print ('2 Phase CW BIM B: %.4f' % ( cos_and_norm_sep(cos_sim_p2_cw_bim_b, grads_p2_cw_bim_b_norms, eta, gamma)))

In [ ]:
plt.plot(np.arange(len(grads_reg_norms[:250])), grads_reg_norms[:250], 'x', label='Original', ms=6, color='black')
plt.plot(np.arange(len(grads_fgsm_norms[:250])), grads_fgsm_norms[:250], 'x', label='FGSM', ms=3)
plt.plot(np.arange(len(grads_bim_a_norms[:250])), grads_bim_a_norms[:250], 'x', label='BIM-A', ms=3)
plt.plot(np.arange(len(grads_bim_b_norms[:250])), grads_bim_b_norms[:250], 'x', label='BIM-B', ms=3, color='red')
plt.plot(np.arange(len(grads_cw_norms[:250])), grads_cw_norms[:250], 'x', label='C&W', ms=3)
plt.plot(np.arange(len(grads_df_norms[:250])), grads_df_norms[:250], 'x', label='DF', ms=3)
plt.plot(np.arange(len(grads_jsma_norms[:250])), grads_jsma_norms[:250], 'x', label='JSMA', ms=3)
#plt.xlabel('Transformed', fontsize=12)
plt.ylabel('Norm', fontsize=12)
plt.title('Norms of Gradient Vectors')
plt.legend(loc="upper right", fancybox=True, shadow=True)
plt.tight_layout()
plt.savefig('mnist_norm.eps', format='eps', dpi=1000)
plt.show()

In [ ]:
plt.plot(np.arange(len(cos_sim_reg[:250])), np.max(cos_sim_reg[:250], axis=1), 'x', label='Original', ms=3, color='black')
plt.plot(np.arange(len(cos_sim_fgsm[:250])), np.max(cos_sim_fgsm[:250], axis=1), 'x', label='FGSM', ms=3)
plt.plot(np.arange(len(cos_sim_bim_a[:250])), np.max(cos_sim_bim_a[:250], axis=1), 'x', label='BIM-A', ms=3)
plt.plot(np.arange(len(cos_sim_bim_b[:250])), np.max(cos_sim_bim_b[:250], axis=1), 'x', label='BIM-B', ms=3, color='red')
plt.plot(np.arange(len(cos_sim_cw[:250])), np.max(cos_sim_cw[:250], axis=1), 'x', label='C&W', ms=3)
plt.plot(np.arange(len(cos_sim_df[:250])), np.max(cos_sim_df[:250], axis=1), 'x', label='DF',ms=3)
plt.plot(np.arange(len(cos_sim_jsma[:250])), np.max(cos_sim_jsma[:250], axis=1), 'x', label='JSMA',ms=3)
#plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
         # fancybox=True, shadow=True, ncol=3)
#plt.xlabel('Transformed', fontsize=12)
plt.ylabel('Cos Sim', fontsize=12)
plt.title('Cosine Similarity of Gradient Vectors')
plt.legend(loc="lower right", fancybox=True, shadow=True)
plt.tight_layout()
plt.savefig('mnist_cos.eps', format='eps', dpi=1000)
plt.show()